In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import os, shutil
import torch
import torch.nn.functional as F
import torchvision.transforms.functional as TF
import torchvision
import time
import pdb

#from sklearn.model_selection import train_test_split
from pathlib import Path
from torch import nn
from torch.utils.data.dataset import Dataset
from torchvision import models, transforms
from collections import defaultdict
from random import shuffle
from PIL import Image
from shapely.geometry import Polygon
import torch.backends.cudnn as cudnn
from tqdm import tqdm_notebook

%matplotlib inline

In [15]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [16]:
data = pd.read_csv('pklot_combined_bbox_rbox.csv')

In [17]:
data.head()

,image_path,occupied,rotated_bbox
0,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 ...,651 636 141 83 -34 705 563 139 77 -44 732 480 ...
1,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 ...,651 636 141 83 -34 705 563 139 77 -44 732 480 ...
2,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ...,651 636 141 83 -34 705 563 139 77 -44 732 480 ...
3,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ...,651 636 141 83 -34 705 563 139 77 -44 732 480 ...
4,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ...,651 636 141 83 -34 705 563 139 77 -44 732 480 ...


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12416 entries, 0 to 12415
Data columns (total 3 columns):
image_path      12416 non-null object
occupied        12416 non-null object
rotated_bbox    12416 non-null object
dtypes: object(3)
memory usage: 291.1+ KB


In [19]:
def rotated_bbox_to_contours(cx, cy, w, h, a):
    """
    Return the four contours for a rotated bounding box
    :param cx: center x coordinate
    :param cy: center y coordinate
    :param w: width of the rectangle
    :param h: height of the rectangle
    :param a: the center angle of the rectangle as degrees
    :return: ((x1, y1), (x2, y2), (x3, y3), (x4, y4))
    """
    theta = a * np.pi / 180 
    dx = w/2
    dy = h/2
    dxcos = dx * np.cos(theta)
    dxsin = dx * np.sin(theta)
    dycos = dy * np.cos(theta)
    dysin = dy * np.sin(theta)
    return (
        np.asarray([cx, cy]) + np.asarray([-dxcos - -dysin, -dxsin + -dycos]),
        np.asarray([cx, cy]) + np.asarray([dxcos - -dysin,  dxsin + -dycos]),
        np.asarray([cx, cy]) + np.asarray([dxcos -  dysin,  dxsin +  dycos]),
        np.asarray([cx, cy]) + np.asarray([-dxcos -  dysin, -dxsin +  dycos])
    )

In [20]:
data['rotated_bbox'] = data['rotated_bbox'].map(lambda x: np.array([int(num) for num in x.split()]))
data['rotated_bbox'] = data['rotated_bbox'].map(lambda x: np.split(x, len(x) / 5))

In [21]:
data['occupied'] = data['occupied'].map(lambda x: [int(num) for num in x.split()])

In [22]:
data['image_path'] = data['image_path'].map(lambda x: Path(x))

In [23]:
data.head()

,image_path,occupied,rotated_bbox
0,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[651, 636, 141, 83, -34], [705, 563, 139, 77,..."
1,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[651, 636, 141, 83, -34], [705, 563, 139, 77,..."
2,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[651, 636, 141, 83, -34], [705, 563, 139, 77,..."
3,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[651, 636, 141, 83, -34], [705, 563, 139, 77,..."
4,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[651, 636, 141, 83, -34], [705, 563, 139, 77,..."


In [24]:
for i in range(data.shape[0]):
    if len(data.loc[i, 'occupied']) != len(data.loc[i, 'rotated_bbox']):
        print('no match for {}!!!'.format(i))

In [25]:
def remove_rotation(bbox):
    cx, cy, h, w, a = bbox
    contours = rotated_bbox_to_contours(cx, cy, w, h, a)
    contours = np.array(contours)
    xmin = np.min(contours[:,0])
    xmax = np.max(contours[:,0])
    
    ymin = np.min(contours[:,1])
    ymax = np.max(contours[:,1])
    
    to_ret = np.array([xmin, ymin, xmax, ymax], dtype=np.int)
    return to_ret

In [26]:
remove_rotation([651, 636, 141, 83, -34])

array([577, 554, 724, 717])

In [27]:
def remove_rotation_list(bboxes):
    non_rotated = []
    for bb in bboxes:
        non_rotated.append(remove_rotation(bb))
    return non_rotated

In [28]:
remove_rotation_list([[651, 636, 141,  83, -34], [705, 563, 139,  77, -44]])

[array([577, 554, 724, 717]), array([629, 486, 780, 639])]

In [29]:
data['rotated_bbox'] = data['rotated_bbox'].map(lambda x: remove_rotation_list(x))

In [30]:
data.head()

,image_path,occupied,rotated_bbox
0,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
1,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
2,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
3,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."
4,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[[577, 554, 724, 717], [629, 486, 780, 639], [..."


In [31]:
image_path_list = list(data['image_path'])
occupied_list = list(data['occupied'])
rotated_bbox_list = list(data['rotated_bbox'])

In [32]:
datalist = {'image_path': [], 'occupied': [], 'rotated_bbox': []}

for i, picp in enumerate(image_path_list):
    
    occupied = ' '.join(map(lambda x: str(x), occupied_list[i]))
    rotated_bbox = ' '.join(map(lambda x: ' '.join(map(lambda y: str(y), x)), rotated_bbox_list[i]))
    
    datalist['image_path'].append(picp)
    datalist['occupied'].append(occupied)
    datalist['rotated_bbox'].append(rotated_bbox)

In [33]:
datalist

{'image_path': [PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_18_35_15.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_18_50_15.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_18_05_14.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_16_35_12.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_17_45_14.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_18_40_15.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_17_50_14.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_19_10_15.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_18_30_15.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_17_00_13.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_17_20_13.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_19_35_16.jpg'),
  PosixPath('PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18_21_15_18.jpg')

In [34]:
data = pd.DataFrame({'image_path': datalist['image_path'], 
                         'occupied': datalist['occupied'],
                         'rotated_bbox': datalist['rotated_bbox']
                        })

In [35]:
data.head()

,image_path,occupied,rotated_bbox
0,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,0 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 ...,577 554 724 717 629 486 780 639 665 416 798 54...
1,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 ...,577 554 724 717 629 486 780 639 665 416 798 54...
2,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ...,577 554 724 717 629 486 780 639 665 416 798 54...
3,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ...,577 554 724 717 629 486 780 639 665 416 798 54...
4,PKLot/PKLot/UFPR04/Rainy/2013-01-18/2013-01-18...,1 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 ...,577 554 724 717 629 486 780 639 665 416 798 54...


In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12416 entries, 0 to 12415
Data columns (total 3 columns):
image_path      12416 non-null object
occupied        12416 non-null object
rotated_bbox    12416 non-null object
dtypes: object(3)
memory usage: 291.1+ KB


In [37]:
data.to_csv('pklot_non_rotated.csv', index=False)